In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from time import time
import io
import re

import pickle
from csv import reader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import layers
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import plot_model

# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from datetime import datetime
from collections import defaultdict

from sklearn.model_selection import train_test_split



In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from transformers import TFXLNetForSequenceClassification, XLNetTokenizer

## Data

In [5]:
test_dataset = pd.read_table("Article-Bias-Prediction-main/data/randomtestdata.csv", sep=',', index_col=0)
test_dataset["pred_labels"] = 4
test_dataset.to_csv("sourcelevel_test_dataset.csv", sep='\t')

In [6]:
start = 0
space = 50
end = space

def interval(s, e):
    return(s+space, e+space)

MAX_LEN = 512
nepochs = 20
nbatch = 16

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tokenizer_transformer = XLNetTokenizer.from_pretrained('xlnet-large-cased')

def create_input_array(df, tokenizer):
    sentences = df.content.values

    input_ids = []
    attention_masks = []
    token_type_ids = []

    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=MAX_LEN,  # Pad & truncate all sentences.
                pad_to_max_length=True,
                return_attention_mask=True,  # Construct attn. masks.
                return_tensors='tf',  # Return tf tensors.
            )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

        token_type_ids.append(encoded_dict['token_type_ids'])

    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    token_type_ids = tf.convert_to_tensor(token_type_ids)


    return input_ids, attention_masks, token_type_ids
#     return input_ids, attention_masks

def convert_inputs_to_tf_dataset(inputs):
    # args.max_seq_len = 256
    ids = inputs[0][0]
    masks = inputs[0][1]
    token_types = inputs[0][2]

    ids = tf.reshape(ids, (-1, MAX_LEN))
    print("Input ids shape: ", ids.shape)
    masks = tf.reshape(masks, (-1, MAX_LEN))
    print("Input Masks shape: ", masks.shape)
    token_types = tf.reshape(token_types, (-1, MAX_LEN))
    print("Token type ids shape: ", token_types.shape)

    ids=ids.numpy()
    masks = masks.numpy()
    token_types = token_types.numpy()
#     return[ids]
#     return [ids, masks, token_types]
    return [ids, masks]



In [7]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:09:00.0, compute capability: 8.6



In [8]:
# this could've been so much easier if .predict wasn't broken.
# Instead, we break up the data into manageable chunks for .predict_on_batch

In [9]:


for i in range(int(1300/space)):
    print(start,end)
    test_dataset = pd.read_table("sourcelevel_test_dataset.csv", index_col=0)
#     test_dataset = pd.read_table("test_dataset.csv",  index_col=0)
    test_section = test_dataset[start:end]
    test_labels = np.asarray(le.fit_transform(test_section['bias_text']))
    test_inputs = [create_input_array(test_section, tokenizer=tokenizer_transformer)]
    test_data = convert_inputs_to_tf_dataset(test_inputs)
    testmodel = TFXLNetForSequenceClassification.from_pretrained("data/experiments/XLNetmodellarge", num_labels=len(np.unique(test_labels)))
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-07, epsilon=1e-08)

    testmodel.compile(optimizer=optimizer, loss=loss, metrics=[metric])
#     test_probs = testmodel.predict_on_batch(test_data, batch_size= 5, verbose=0)
    test_probs = testmodel.predict_on_batch(test_data)
    test_probs = test_probs["logits"]
    test_classes = np.argmax(test_probs, axis = 1)
    c=0
    for i in range(test_classes.shape[0]):
        if test_classes[i]==test_labels[i]:
            c+=1
    print("Test accuracy:")
    print(c/test_labels.shape[0])
    test_dataset["pred_labels"][start:end] = test_classes
    test_dataset.to_csv("sourcelevel_test_dataset.csv", sep='\t')
    
    start, end = interval(start,end)
    tf.keras.backend.clear_session()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 50


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2104: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Input ids shape:  (50, 512)
Input Masks shape:  (50, 512)
Token type ids shape:  (50, 512)


All model checkpoint layers were used when initializing TFXLNetForSequenceClassification.

All the layers of TFXLNetForSequenceClassification were initialized from the model checkpoint at data/experiments/XLNetmodellarge.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetForSequenceClassification for predictions without further training.


...

Test accuracy:
0.88
1200 1250
Input ids shape:  (50, 512)
Input Masks shape:  (50, 512)
Token type ids shape:  (50, 512)


All model checkpoint layers were used when initializing TFXLNetForSequenceClassification.

All the layers of TFXLNetForSequenceClassification were initialized from the model checkpoint at data/experiments/XLNetmodellarge.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetForSequenceClassification for predictions without further training.


Test accuracy:
0.84
1250 1300
Input ids shape:  (50, 512)
Input Masks shape:  (50, 512)
Token type ids shape:  (50, 512)


All model checkpoint layers were used when initializing TFXLNetForSequenceClassification.

All the layers of TFXLNetForSequenceClassification were initialized from the model checkpoint at data/experiments/XLNetmodellarge.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetForSequenceClassification for predictions without further training.


Test accuracy:
0.84


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
# # accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(test_labels, test_classes)
# print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(test_labels, test_classes)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(test_labels, test_classes)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(test_labels, test_classes)
# print('F1 score: %f' % f1)
 

In [ ]:
confusion_matrix(test_labels, test_classes)

In [ ]:
# train_section.to_csv("train_dataset.csv", sep='\t')
# val_section.to_csv("val_dataset.csv", sep='\t')
# test_section.to_csv("test_dataset.csv", sep='\t')

In [ ]:
# model.save_pretrained(save_path)

In [ ]:
tf.keras.backend.clear_session()